In [ ]:
import numpy as np
from scipy.integrate import quad
import scipy.optimize as scpo
import numpy as np
import pandas as pd
from geneticalgorithm import geneticalgorithm as ga

In [ ]:
call_df = pd.read_csv('')
History = pd.read_csv('')

In [ ]:
def Heston_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v) ** 2 -
                  sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) \
        / (kappa_v - rho * sigma_v * u * 1j - c2)
    H1 = (r * u * 1j * T + (c1 / sigma_v ** 2) *
          ((kappa_v - rho * sigma_v * u * 1j + c2) * T -
          2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v ** 2 *
          ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

def Heston_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    char_func_value = Heston_char_func(u - 1j * 0.5, T, r, kappa_v,
                                    theta_v, sigma_v, rho, v0)
    int_func_value = 1 / (u ** 2 + 0.25) \
        * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    return int_func_value

def Heston_call_value(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    int_value = quad(lambda u:
                     Heston_int_func(u, S0, K, T, r, kappa_v,
                                  theta_v, sigma_v, rho, v0),
                     0, np.inf, limit=250)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) /
                     np.pi * int_value)
    return call_value

f2 = np.vectorize(Heston_call_value) #Vectorize the Heston call pricer

In [ ]:
def Heston_paths(S0, mu, v0, rho, kappa, theta, xi, T, dt):
    MU  = np.array([0, 0])
    COV = np.matrix([[1, rho], [rho, 1]])
    W   = np.random.multivariate_normal(MU, COV, T)
    W_S = W[:,0]
    W_v = W[:,1]

    # Generate paths
    vt    = np.zeros(T)
    vt[0] = v0
    St    = np.zeros(T)
    St[0] = S0
    for t in range(1,T):
        vt[t] = np.abs(vt[t-1] + kappa*(theta-np.abs(vt[t-1]))*dt + xi*np.sqrt(np.abs(vt[t-1]))*W_v[t]*np.sqrt(dt))
        St[t] = St[t-1]*np.exp((mu - 0.5*vt[t-1])*dt + np.sqrt(vt[t-1]*dt)*W_S[t])

    return St

In [ ]:
def Heston_call(row): # Heston pricer for a df
    S0=row.Closing_price
    K=row.stike
    T=row.nDiff/365
    r=row.r/100
    kappa_v=row.kappa_v
    theta_v=row.theta_v
    sigma_v=row.sigma_v
    rho=row.rho
    v0=row.v0
    u=100
    
    int_value = quad(lambda u:
                     Heston_int_func(u, S0, K, T, r, kappa_v,
                                  theta_v, sigma_v, rho, v0),
                     0, np.inf, limit=250)[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) /
                     np.pi * int_value)
    return call_value

# Non-linear LS Calibration

In [ ]:
def Heston_function(x,kappa_v, theta_v, sigma_v, rho, v0): #Formated for calibration
    K, S0, r, T = x
    r = r/100
    T = T/365
    u = 10
    return f2(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)

In [ ]:
init_vals = [0.3, 0.1, 0.2, 0.1, 0.4]
bounds = ([0.05, 0.05, 0.05, -0.9, 0.05] , [1,1,1,1,1])
params_Heston = scpo.curve_fit(Heston_function, (call_df.stike.values,call_df.Closing_price.values,call_df.r.values,call_df.nDiff.values),call_df.Close.values, p0=init_vals, bounds=bounds)

In [ ]:
call_df[['kappa_v','theta_v','sigma_v','rho','v0']] = params_Heston[0]

In [ ]:
Heston_params = call_df[['kappa_v','theta_v','sigma_v','rho','v0']].iloc[0]

Heston_params.to_csv('./Heston_params.csv') #Save parameters

In [ ]:
call_df['Heston'] = call_df.apply(Heston_call, axis=1)

# Returns for model with Non-lienar LS calibrated params

In [ ]:
S0= History[0].close
kappa,theta, xi, rho, v0 = Heston_params
T = len(History)
dt = 1/365
MU = np.array([0, 0])
COV = np.matrix([[1, rho], [rho, 1]])
W = np.random.multivariate_normal(MU, COV, T)
W_S = W[:,0]
W_v = W[:,1]
mu = 0
# Generate paths
vt = np.zeros(T)
vt[0] = v0
St = np.zeros(T)
St[0] = S0
for t in range(1,T):
    vt[t] = np.abs(vt[t-1] + kappa*(theta-np.abs(vt[t-1]))*dt + xi*np.sqrt(np.abs(vt[t-1]))*W_v[t]*np.sqrt(dt))
    St[t] = St[t-1]*np.exp((mu - 0.5*vt[t-1])*dt + np.sqrt(vt[t-1]*dt)*W_S[t])

St.to_csv('./Heston-stock')

# GA Calibration

In [ ]:
varbound=np.array([[0.01,1],[0.01,1],[0.01,1],[-1,1],[0.01,1]])
algorithm_param = {'max_num_iteration': 3000,\
            'population_size':400,\
            'mutation_probability':0.1,\
            'elit_ratio': 0.01,\
            'crossover_probability': 0.5,\
            'parents_portion': 0.3,\
            'crossover_type':'uniform',\
            'max_iteration_without_improv':23}

In [ ]:
def Heston_GA(y):
   def f(x):
    S0= y [0]
    kappa,theta, xi, rho, v0 = x
    T = len(y)
    dt = 1/365
    MU = np.array([0, 0])
    COV = np.matrix([[1, rho], [rho, 1]])
    W = np.random.multivariate_normal(MU, COV, T)
    W_S = W[:,0]
    W_v = W[:,1]
    mu = 0
    # Generate paths
    vt = np.zeros(T)
    vt[0] = v0
    St = np.zeros(T)
    St[0] = S0
    for t in range(1,T):
        vt[t] = np.abs(vt[t-1] + kappa*(theta-np.abs(vt[t-1]))*dt + xi*np.sqrt(np.abs(vt[t-1]))*W_v[t]*np.sqrt(dt))
        St[t] = St[t-1]*np.exp((mu - 0.5*vt[t-1])*dt + np.sqrt(vt[t-1]*dt)*W_S[t])
    
    St.to_csv('./Heston-stock-GA')
    return np.mean((St-y)**2)
   model = ga(function=f,\
            dimension=5,\
            variable_type='real',\
            variable_boundaries=varbound,\
            algorithm_parameters=algorithm_param,
         convergence_curve=False,
         progress_bar=True)

   model.run()
   return model.best_variable

In [ ]:
call_df[['mu', 'kappa','theta', 'sigma_v', 'rho', 'v0']] = Heston_GA(History.Close.values)

In [ ]:
call_df['Heston-GA'] = call_df.apply(Heston_call, axis=1)

# Save dataframe

In [ ]:
call_df.to_csv('./Heston-input-output')

# Metrics

In [ ]:
from .utilties import utilties

In [ ]:
line1 = utilties.error_metrics(call_df['Close'],call_df['Heston'])

In [ ]:

line2 = utilties.error_metrics(call_df['Close'],call_df['Heston-GA'])

In [ ]:
for line in ([*line1], [*line2]):
  print('& {:.2f} & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% \\\\'.format(*line))